In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import interact
from tqdm.notebook import tqdm

In [2]:
from lucky_investor import luckyInvestor
from unlucky_investor import unluckyInvestor
from som_investor import somInvestor
from random_investor import randomInvestor

In [3]:
df = pd.read_csv('tadawul_data30.csv',parse_dates=['date'])
df = df[df['price'] !=0]

In [4]:
#df['date'] = pd.to_datetime(df['transactionDateStr'])
#df = df[['symbol','previousClosePrice','date']].rename(columns={'previousClosePrice':'price'})

In [5]:
# @interact(symbol = df.groupby(['symbol'])['price'].count().sort_values().tail(30).index)

# def chooseSymbol(symbol):
    
#     lucky_investor = luckyInvestor('lucky',100_0000)
#     unlucky_investor = unluckyInvestor('unlucky',100_0000)
#     som_investor = somInvestor('som',100_0000)
    
#     dfs = df[df['symbol'] == symbol]
#     dfs = dfs.sort_values('date').reset_index(drop=True)
#     dfs['YM'] = dfs['date'].apply(lambda v: v.strftime('%Y-%m'))

#     for ym in np.sort(dfs['YM'].unique()):
#         dfym = dfs[dfs['YM'] == ym]
#         lucky_investor.step(dfym)
#         unlucky_investor.step(dfym)
#         som_investor.step(dfym)

#     plt.figure(figsize=(10,8))

#     plt.plot(lucky_investor.portfolio,label='lucky')
#     plt.plot(unlucky_investor.portfolio,label='unlucky')
#     plt.plot(som_investor.portfolio,label='som')
#     plt.legend()
#     plt.grid()
#     plt.show()

#     for investor in investors:
#         print(investor)
#         print("*"*60)

In [6]:
symbols30 = df.groupby(['symbol'])['price'].count().sort_values().tail(30).index

In [7]:
symbols = symbols30#list(symbols30[20:30])

In [ ]:
initial_deposit = 0
shares_to_buy = 5
amount_to_deposit_monthly = 1_000

lucky_investor = luckyInvestor('lucky',initial_deposit)
unlucky_investor = unluckyInvestor('unlucky',initial_deposit)
som_investor = somInvestor('som',initial_deposit)
random_investor = randomInvestor('random',initial_deposit)
investors = [lucky_investor,unlucky_investor,som_investor,random_investor]

dfs = df[df['symbol'].apply(lambda symbol: symbol in symbols)]
dfs = dfs.sort_values('date').reset_index(drop=True)
dfs['YM'] = dfs['date'].apply(lambda v: v.strftime('%Y-%m'))
yms = np.sort(dfs['YM'].unique())
pbar = tqdm(yms)
for ym in pbar:
    pbar.set_description("Processing %s" % ym)
    dfym = dfs[dfs['YM'] == ym]
    
    for investor in investors:
        investor.depositMoney(amount_to_deposit_monthly)
    for symbol in symbols: 
        dfym_symbol = dfym[dfym['symbol'] == symbol]
        for investor in investors:
            investor.buyLogic(dfym_symbol,shares=shares_to_buy)
    
    symbols_and_prices = dfym.groupby(['symbol'],as_index=False)['date'].max()
    symbols_and_prices = symbols_and_prices.merge(dfym,on=['symbol','date'])
    stocks = {}
    for i,(symbol,price,date) in symbols_and_prices[['symbol','price','date']].iterrows():
        stocks[int(symbol)] = price
        stocks['date'] = date
        
    for investor in investors:
        investor.updatePortfolio(stocks)
        investor.calculateProfit(stocks)

        
        
fig = go.Figure()

for investor in investors:
    fig.add_trace(go.Scatter(x=yms, y=investor.portfolio,
                             name=investor.name))

fig.update_layout(hovermode='x unified')
fig.show()   

# for investor in investors:
#     print(investor)
#     print("*"*60)

  0%|          | 0/256 [00:00<?, ?it/s]

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
df[df['price']==0]

In [ ]:
investor.stock_profit[investor.stock_profit['avg_cost']==0]

In [ ]:
investors_profit = pd.concat([investor.computeAnnualReturn() for investor in investors])

In [ ]:
investors_profit2 = pd.DataFrame()
for investor in investors:
    profit = investor.profit.copy()
    profit.insert(0,'name',investor.name)
    investors_profit2 = pd.concat([investors_profit2,profit])
investors_profit2['year'] = investors_profit2['YM'].apply(lambda v: v[:4])

In [ ]:
fig = px.line(investors_profit2.groupby(['year','name'],as_index=False)['profit pct'].mean(),
        x='year',y='profit pct',color='name')

fig.update_layout(hovermode='x unified')

In [ ]:
pd.crosstab(investors_profit2['year'],
            investors_profit2['name'],
            investors_profit2['profit pct']*100,
            aggfunc='mean',margins=True,margins_name='mean')[[investor.name for investor in investors]]\
  .style.background_gradient(axis=1).format('{:.1f}')

In [ ]:
# investors_profit.head(500)

In [ ]:
investor.transaction_history

In [ ]:
investor.transaction_history.groupby(['YM'],as_index=False).agg({'total':'sum',
                                                                 'shares':'sum',
                                                                 'symbol':lambda v:'-'.join(map(str,v))});

In [ ]:
for investor in investors:
    print(investor)
    print("**"*30)

In [ ]:
fig = go.Figure()

for investor in investors:
    fig.add_trace(go.Scatter(x=investor.profit.YM, y=investor.profit['profit pct'],
                             name=investor.name))

fig.update_layout(hovermode='x unified')
fig.show()   